In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import yeojohnson
from scipy.stats import skew
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
from scipy.stats import normaltest

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold, cross_validate, learning_curve, RepeatedKFold
from sklearn.linear_model import BayesianRidge
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, FunctionTransformer, RobustScaler, StandardScaler
from sklearn.linear_model import BayesianRidge, Lasso
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import shapiro
from mapie.regression import MapieRegressor
from mapie.metrics import regression_coverage_score


import scipy.stats as ss
from scipy.stats import boxcox

import math 
import numpy as np
from scipy import stats

import os
import re
import timeit
import re

import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline

import pickle

from mapie.metrics import regression_coverage_score

fig_size = plt.rcParams["figure.figsize"]

pd.options.display.float_format = '{:.9f}'.format


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Functions

In [6]:
def fix_col_names(df):
    '''
    takes a df and converts column names to lowercase underscore seperated
    '''
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

In [7]:
def summary(df,data_type='numeric'):
    '''
    Calcualte the descriptive stats for different type of data
        Args:
            df: DataFrame.
            data_type=['numeric','categorical']
        Returns:
            this function returns a data frame containing summary stats for different types of data
                

    '''
    df_column=df.select_dtypes(include=['int64','float64']).columns
    df_describe = df[df_column]
    stats = {
    "Variable": list(x.title() for x in df_column),
    "Min": list(df_describe.apply(np.min,axis=0).values),
    "Median": list(df_describe.apply(np.median,axis=0).values),
    "Mean": list(df_describe.apply(np.mean,axis=0).values),
    "Variance": list(df_describe.apply(np.var,axis=0).values),
    "Max": list(df_describe.apply(np.max,axis=0).values),
    "Std": list(df_describe.apply(np.std,axis=0).values),
    "Kurtosis": df_describe.apply(lambda x:x.kurt(),axis=0).values.tolist(),
    "Skewness": df_describe.apply(lambda x:x.skew(),axis=0).values.tolist(),
    # "Sum": list(df_describe.apply(np.sum,axis=0).values),
    # "Mad": df_describe.apply(lambda x:x.mad(),axis=0).values.tolist(),
    "N_Zeros": df_describe.apply(lambda x:len(x)-np.count_nonzero(x),axis=0).values.tolist(),
    "N_Nulls": df_describe.apply(lambda x:np.count_nonzero(np.isnan(x)),axis=0).values.tolist(),
    'Count': df_describe.apply(lambda x:len(x),axis=0).values
    }
    print ("Numeric Variables Dataset Shape: "+ str(df_describe.shape))
    return pd.DataFrame(stats)

In [8]:
def display_descriptive_stats(df_in, col_ls, nfeats_per_group=25):
    """
    This function displays descriptive statistics for the specified columns/list of columns. It splits the specified columns into groups and displays 
    the statistics for each group side by side.
    
    Parameters:
    df_in (pd.DataFrame): The input DataFrame containing the data for which descriptive statistics are to be displayed.
    col_ls (list): A list of column names for which the descriptive statistics are to be calculated and displayed.
    nfeats_per_group (int, optional): The number of features to display per group. Default is 25.
    
    Returns:
    None: The function displays the results and does not return any value.
    
    The function performs the following steps:
    1. Prints the count of columns to be analyzed.
    2. Iterates over each group of columns, calculates descriptive statistics for each group, and displays the 
    statistics in three categories:
    - General statistics including Min, Median, Mean, Max, Kurtosis, and Skewness.
    - Standard Deviation.
    - Count statistics including the number of zeros, number of nulls, and total count.
    5. Uses pandas Styler to format the DataFrames and apply color gradients to enhance visual interpretation.
    """
    
    print('cols cnt for analysis:  ', len(col_ls))
    
    def horizontal(dfs):
        from IPython.display import HTML
        
        html = '<div style="display:flex">'
        for df in dfs:
            html += '<div style="margin-right: 32px">'
            html += df.to_html()
            html += '</div>'
        html += '</div>'
        display(HTML(html))
    
    loop_cnt = range(int(np.ceil(len(col_ls)/nfeats_per_group)))
    print('loop cnt: ',loop_cnt)
    print('feature count per table: ',nfeats_per_group)

    
    n=0
    for i in loop_cnt:
        
        tbl_cols = col_ls[n:n+nfeats_per_group]

        
        df_num_stats = summary(df_in[tbl_cols])

        df_stat_points = df_num_stats[['Variable','Min', 'Median', 'Mean', 'Max', 'Kurtosis', 'Skewness']]
        # df_stat_points.index = df_num_stats['Variable']
        df_stat_std = df_num_stats[['Std']]
        df_stat_cnts = df_num_stats[['N_Zeros',	'N_Nulls', 'Count']]
        
        # display(df_num_stats_summary, '\n')
        
        # df_time_med = df_time_pivot.loc[:, pd.IndexSlice['median',:]]
        # df_time_mean = df_time_pivot.loc[:, pd.IndexSlice['mean',:]]
       
        horizontal(

            # https://stackoverflow.com/questions/38931566/pandas-style-background-gradient-both-rows-and-columns
            # df_stat_points - might benefit from all on the same scale, but there's no direct option, this link gives a possible solution
            # other option is to have coloring follow a log scale to remove influence of outliers
            
           [df_stat_points.style.format(precision=2,thousands=',').background_gradient(cmap='PuBu',axis=1),
            df_stat_std.style.format(precision=2,thousands=',').hide(axis="index").background_gradient(cmap='inferno',axis=1),
            df_stat_cnts.style.format(precision=2,thousands=',').hide(axis="index").background_gradient(cmap='BuGn',axis=1)]
            # raw=True
        )

        n+=nfeats_per_group



In [9]:
def check_normality_and_style(df, num_ls):
    """
    Checks the normality of columns in a DataFrame and returns a styled DataFrame
    highlighting columns that are not normally distributed.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    num_ls (list): List of column names to check for normality.

    Returns:
    pd.io.formats.style.Styler: A styled DataFrame with normality test results.
    """
    # Create a list to store the results
    results = []

    # Loop through the columns and perform the normality test
    for i in num_ls:
        stat, p_value = normaltest(df[i].values)
        label = "Not Normal" if p_value >= 0.05 else "Normal"
        results.append([i, label, stat, p_value])

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results, columns=['Column', 'Normality', 'Statistic', 'p-value'])

    # Function to highlight Gaussian rows
    def highlight_gaussian(row):
        color = 'lightblue' if row['Normality'] == 'Not Normal' else ''
        return ['background-color: {}'.format(color) for _ in row]

    # Apply the highlighting function to the DataFrame
    styled_results_df = results_df.style.apply(highlight_gaussian, axis=1)

    return styled_results_df

In [10]:
def plot_jointplots(df, num_ls, target):
    """
    Generates joint plots for each numerical feature in the list against the target variable,
    including both the original and Yeo-Johnson transformed features.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    num_ls (list): List of numerical column names to plot.
    target (str): The target variable name.
    
    """
    for variable in num_ls:
        print("_" * 30)
        print(variable)
        
        # Normal feature plot (g0)
        g0 = sns.jointplot(x=df[variable], y=df[target], kind='reg')
        g0.savefig('g0.png')
        plt.close(g0.fig)

        try:
            # Yeo-Johnson transformed feature plot (g1)
            transformed_variable, _ = yeojohnson(df[variable])
            g1 = sns.jointplot(x=transformed_variable, y=df[target], kind='reg')
            g1.savefig('g1.png')
            plt.close(g1.fig)
        except Exception as e:
            print(e)
            g1 = None

        # Create subplots from temporal images
        f, axarr = plt.subplots(ncols=2, sharey=False, figsize=(25, 30))
        axarr[0].imshow(mpimg.imread('g0.png'))
        if g1:
            axarr[1].imshow(mpimg.imread('g1.png'))
        else:
            axarr[1].text(0.5, 0.5, 'Transformation Error', 
                          horizontalalignment='center', 
                          verticalalignment='center')

        # Turn off x and y axis
        [ax.set_axis_off() for ax in axarr.ravel()]

        plt.tight_layout()
        plt.show()

        # Clean up the temporary image files
        os.remove('g0.png')
        if g1:
            os.remove('g1.png')

In [ ]:
# # Change the data path and the target if neccesary
# DATPATH = r'C:\Users\rhead\Documents\local_ds\Pricing\\'

# target = 'avg_single_game_ticket_price'

In [ ]:
# #Replace the name of the csv here
# dat = fix_col_names(pd.read_csv(DATPATH+r'Regression Model Data 2023.csv').set_index('Team'))

# # Output of this will specify (rows, columns) of the dataset
# dat.shape

# EDA

In [ ]:
# Analyze the dtypes of the data set and make a note of all that migh trequire a change in dtypes
dat.info(verbose=1,show_counts=True)

In [ ]:
#dataframe to only include numberic columns for now
df_column=dat.select_dtypes(include=['int64','float64']).columns
df = dat[df_column]

In [ ]:
#Correct the variable names. Remove parantheses and trailing underscores
print(df.columns)
df = remove_brackets_from_column_names(df)
print("\nDataFrame after removing brackets from column names:")
print(df.columns)

## Encode and Analyze the target

The target is transformed using boxcox transformation to analyze the normal distribution with the other independent variables. Transforming it via boxcox for EDA allows to easily interpret the relationship between the target and variables

In [ ]:
#Visualize the target and transform it

df['tranformed_target'], _ = boxcox(df[target])

fig,ax = plt.subplots(1,3,figsize=(20,5))
df['tranformed_target'].plot(kind="hist",ax=ax[0])
df['tranformed_target'].plot(kind="kde",ax=ax[1])
df['tranformed_target'].plot(kind="box",ax=ax[2])
plt.show()
print(f'{target}: {"Not Gaussian" if normaltest(df[target].values,)[1] < 0.05 else "Gaussian"}  {normaltest(df[target].values)}')

In [ ]:
#Check for unique values. Get rid of constants in the model
df.nunique().sort_values()

## Data type lists

In [ ]:
#num_columns
num_ls = ['capacity',
          'cost_of_living_index',
          'per_capita_personal_income',
          'city_population',
          'metro_population',
          'venue_distance_from_city_center',
          '22-23_win_%',
          '3yr_win_%',
          '5_yr_win_%',
          'attendance_rate',
          'age_of_venue',
          'age_of_club',
          'forbes_valuation_2022',
          'forbes_valuation',
          'allstars_roster',
          'allstars_div_roster',
          'allstars_current',
          'allstars_div_current',
          'in_market_teams',
          'former_mvp',
          'size_of_dma',
          'household_income',
          'population_growth_',
          'no_of_fortune_500_companies',
          'average_money_spent_on_sports',
          'job_posting_activity',
          'student_population_percent',
          'median_home_value',
          'per_cap_inc_chg']
#boolean columns to be treated separately
bool_ls = [
    'most_followed_team',
    'most_watched_sport',
    'allstars_current_flag',
    'allstars_roster_flag',
    ]

In [ ]:
#this list should have all the columns not required for modeling
set(df)-set(num_ls)-set(bool_ls)

## Display descreptive stats

In [ ]:
#Compare median and means. if there is a big difference probability is there for outliers. Check for number of zeroes and nulls in the data
display_descriptive_stats(df, num_ls, nfeats_per_group=20)

## Check for Normality

Because we have so many distributions that are not normal we will have to check for relation with the target using Yeo-Johnson transformation of the variable

In [ ]:
styled_results = check_normality_and_style(df, num_ls)
styled_results

## Visualize relationship with the target

In [ ]:
#Use these graphs to see the relationship between the target and the feature. The graphs on the right shows the transformed feature with the target. The histograms on each side represent
# the distribution of the fetures. Yeo-Johnson transformation will transform non-normal distribution to normal distribution to understand the relationship with the dependent variable
plot_jointplots(df, num_ls, 'tranformed_target')

In [ ]:
# ---------------------End of EDA------------------------------ #

# Regression Modeling

## Functions

In [ ]:
# Function to check normality
def check_normality(X):
    return np.array([shapiro(X[:, i])[1] > 0.05 for i in range(X.shape[1])])

# Custom transformer to apply Yeo-Johnson only to non-normally distributed columns
class ConditionalTransformer(PowerTransformer):
    def fit(self, X, y=None):
        self.normality_mask_ = check_normality(X)
        return super().fit(X[:, ~self.normality_mask_], y)

    def transform(self, X):
        X_transformed = X.copy()
        if hasattr(self, 'normality_mask_'):
            X_transformed[:, ~self.normality_mask_] = super().transform(X[:, ~self.normality_mask_])
        return X_transformed

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

def cross_val_tune_regressor(dfX_train, dfy_train, num_ls, bool_ls, Regressor, param_grid):
    # Define numerical and boolean pipelines
    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')), 
        ('robust', RobustScaler()),
        ('yeojohnson', FunctionTransformer())  # Placeholder for ConditionalTransformer
    ])

    bool_pipeline = Pipeline([
        ('identity', FunctionTransformer())
    ])

    # Combine pipelines with ColumnTransformer
    preprocessor = ColumnTransformer([
        ('num', num_pipeline, num_ls),
        ('bool', bool_pipeline, bool_ls)
    ])

    # Combine preprocessor with model in a pipeline
    model_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', Regressor())
    ])
#RD: try repeated k fold. 
    # Initialize KFold
    kf = KFold(n_splits=10, shuffle=True, random_state=0)

    # Lists to store metrics and predictions
    in_sample_metrics = []
    out_of_sample_metrics = []
    out_of_sample_predictions = np.zeros_like(dfy_train)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=model_pipeline, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1, error_score='raise')

    # Perform KFold cross-validation -- score
    for i, (train_index, test_index) in enumerate(kf.split(dfX_train)):
        X_train, X_test = dfX_train.iloc[train_index], dfX_train.iloc[test_index]
        y_train, y_test = dfy_train.iloc[train_index], dfy_train.iloc[test_index]

        # Fit GridSearchCV
        grid_search.fit(X_train, y_train)

        # Best parameters
        print("Best parameters found: ", grid_search.best_params_)

        # Best estimator
        best_model = grid_search.best_estimator_

        # Predict
        y_train_pred = best_model.predict(X_train)
        y_test_pred = best_model.predict(X_test)

        # Calculate metrics for in-sample
        in_sample_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        in_sample_mae = mean_absolute_error(y_train, y_train_pred)
        in_sample_r2 = r2_score(y_train, y_train_pred)
        in_sample_metrics.append((in_sample_rmse, in_sample_mae))

        # Calculate metrics for out-of-sample
        out_of_sample_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        out_of_sample_mae = mean_absolute_error(y_test, y_test_pred)
        out_of_sample_r2 = r2_score(y_test, y_test_pred)
        out_of_sample_metrics.append((out_of_sample_rmse, out_of_sample_mae))

        # Collect out-of-sample predictions
        out_of_sample_predictions[test_index] += y_test_pred

    # Calculate average metrics
    avg_in_sample_metrics = np.mean(in_sample_metrics, axis=0)
    avg_out_of_sample_metrics = np.mean(out_of_sample_metrics, axis=0)

    # Print average metrics
    print("Average In-Sample Metrics (RMSE, MAE):", avg_in_sample_metrics)
    print("Average Out-of-Sample Metrics (RMSE, MAE):", avg_out_of_sample_metrics)

    # Scatter plot of actuals vs. average out-of-sample predictions
    plt.scatter(dfy_train, out_of_sample_predictions)
    plt.xlabel("Actual values")
    plt.ylabel("Out-of-sample predictions")
    plt.title("Scatter plot of Actuals vs. Out-of-Sample Predictions")
    return plt, out_of_sample_predictions, best_model

def plot_actual_vs_predicted(model, dfy_train, out_of_sample_predictions):
    # Apply exponential transformation
    y_pred = np.exp(out_of_sample_predictions)
    y_train = np.exp(dfy_train)
    
    true_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
    true_mae = mean_absolute_error(y_train, y_pred)

    print("True RMSE:", true_rmse)
    print("True MAE:", true_mae)
    
    # Create a DataFrame for scatter plot
    scatter_data = pd.DataFrame({
        'actual ticket price': y_train.values,
        'predicted ticket price': y_pred
    }, index = list(dfy[dfy.index!='MIN'].index))
    
    scatter_data.index.name = 'Team'

    new_data = {'actual ticket price': np.exp(dfy_test).values[0], 'predicted ticket price': np.exp(model.predict(dfX_test))[0]}
    scatter_data.loc['MIN'] = new_data
    
    # Define indices for outliers and test set
    outliers_indices = ['NYK', 'LAL', 'GSW']  # Example indices for outliers
    test_set_index = 'MIN'  # Example index for the test set
    
    # Create a new column in scatter_data for colors
    scatter_data['color'] = 'blue'
    scatter_data.loc[outliers_indices, 'color'] = 'red'
    scatter_data.loc[test_set_index, 'color'] = 'green'
    
    # Extract x, y, and labels
    x = scatter_data['actual ticket price']
    y = scatter_data['predicted ticket price']
    labels = scatter_data.index
    colors = scatter_data['color']
    
    # Create the scatter plot with seaborn
    fig, ax = plt.subplots(1, figsize=(15, 8))
    fig.suptitle('Actual v Predicted Ticket Price')
    
    # Plot the scatter points with the respective colors
    sns.scatterplot(x=x, y=y, hue=colors, palette={'blue': 'lightblue', 'red': 'red', 'green': 'green'}, ax=ax, s=50, edgecolor='k', alpha=0.7, legend=False)
    
    # Add a regression line
    sns.regplot(x="actual ticket price", y="predicted ticket price", data=scatter_data, scatter=False, ax=ax, color='grey')
    
    # Annotate the points with team names
    for i, txt in enumerate(labels):
        ax.annotate(txt, (x[i], y[i]), xytext=(0, 5), textcoords='offset points')
    
    # Show the plot
    plt.show()
    
    return plt, scatter_data

## Correlations Check

In [ ]:
corr_matrix = df.corr()

In [ ]:
plt.figure(figsize=(20, 15))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Correlation Heatmap')
plt.show()

## Train/Test Split

Because we only have 30 obs, in order to test our model we will leave out only 1 record as a hold out set

In [ ]:
dfX, dfy = df.drop(target,axis=1), np.log(df[target])
print(dfX.shape)

print('num_attribs: ',len(num_ls))
print('cat_attribs: ',len(bool_ls))

#======== train test split =========== manual split. 1 record holdout
# dfX_train, dfX_test, dfy_train, dfy_test = train_test_split(dfX, dfy, test_size=0.2, random_state=123)
dfX_train = dfX[dfX.index!= 'MIN']
dfX_test = dfX[dfX.index == 'MIN']
dfy_train = dfy[dfy.index!= 'MIN']
dfy_test = dfy[dfy.index == 'MIN']

dfX_train = dfX_train.reset_index()
dfX_test = dfX_test.reset_index()
dfy_train = dfy_train.reset_index()
dfy_test = dfy_test.reset_index()

#reorder cats to the back
dfX_train = dfX_train[num_ls+bool_ls]
dfX_test = dfX_test[num_ls+bool_ls]

dfy_train = dfy_train[target]
dfy_test = dfy_test[target]


In [ ]:
print('_____________________________')
print('train / test')
print(dfX_train.shape,'/',dfX_test.shape)
print(dfy_train.shape,'/',dfy_test.shape)

## Train the regressor
The output of this cell will be the best alpha found for all CVs. Average in sample and out of sample metrics and the scatter plot for in sample and out of sample predictions

In [ ]:
# Regressor: The name of the regression algorithm 
Regressor = Lasso
# param_grid (dictionary): list out all the hyperparameters for the algorithm selected and the respective range for them
param_grid = {'regressor__alpha': np.logspace(-4, 0, 50)}
plt, out_of_sample_predictions, model = cross_val_tune_regressor(dfX_train, dfy_train, num_ls, bool_ls, Regressor, param_grid)

## Plot actual values of the target using scatter plot

In [ ]:
plt, scatter_data = plot_actual_vs_predicted(model, dfy_train, out_of_sample_predictions)

# Calculate prediction intervals
to calculate prediction intervals fro each record with 95% confidence to get a better view of the redictions from the model

In [ ]:
def rep_cross_val_tune_regressor(dfX_train, dfy_train, num_ls, bool_ls, Regressor, param_grid):
    """
    Perform repeated cross-validation for hyperparameter tuning and evaluation of a regression model to come up with prediction intervals
    
    Parameters:
    - dfX_train (pd.DataFrame): Training features data.
    - dfy_train (pd.Series or pd.DataFrame): Training target data.
    - num_ls (list): List of indices or column names for numerical features.
    - bool_ls (list): List of indices or column names for boolean features.
    - Regressor (sklearn estimator): Regressor class to be used.
    - param_grid (dict): Dictionary with parameters names and lists of parameter settings to try.
    
    Returns:
    - out_of_sample_predictions_df (pd.DataFrame): DataFrame containing out-of-sample predictions for each fold.
    - in_sample_metrics (list): List of tuples containing in-sample RMSE and MAE for each fold.
    - out_of_sample_metrics (list): List of tuples containing out-of-sample RMSE and MAE for each fold.
    """
    
    # Define numerical and boolean pipelines
    num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), 
    ('robust', RobustScaler()),
    ('yeojohnson', FunctionTransformer())  # Placeholder for ConditionalTransformer
    ])

    bool_pipeline = Pipeline([
        ('identity', FunctionTransformer())
    ])

    # Combine pipelines with ColumnTransformer
    preprocessor = ColumnTransformer([
        ('num', num_pipeline, num_ls),
        ('bool', bool_pipeline, bool_ls)
    ])

    # Combine preprocessor with model in a pipeline
    model_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', Regressor())
    ])

    # Initialize KFold
    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=0)

    num_folds = 10 * 10  # 10 splits * 10 repeats
    out_of_sample_predictions_df = pd.DataFrame(np.zeros((dfX_train.shape[0], num_folds)))

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=model_pipeline, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1, error_score='raise')

    # Perform KFold cross-validation -- score
    for fold_idx, (train_index, test_index) in enumerate(rkf.split(dfX_train)):
        X_train, X_test = dfX_train.iloc[train_index], dfX_train.iloc[test_index]
        y_train, y_test = dfy_train.iloc[train_index], dfy_train.iloc[test_index]

        # Fit GridSearchCV
        grid_search.fit(X_train, y_train)

        # Best parameters
        # print("Best parameters found: ", grid_search.best_params_)

        # Best estimator
        best_model = grid_search.best_estimator_

        # Predict
        y_train_pred = best_model.predict(X_train)
        y_test_pred = best_model.predict(X_test)

        out_of_sample_predictions_df.iloc[test_index, fold_idx] = y_test_pred

    out_of_sample_predictions_df.columns = [f'y_pred{i+1}' for i in range(num_folds)]

    #get upper, lower and median from the above table for each record to create prediction intervals
    out_of_sample_predictions_df['lower'] = out_of_sample_predictions_df[out_of_sample_predictions_df > 0].min(axis=1)
    out_of_sample_predictions_df['upper'] = out_of_sample_predictions_df[out_of_sample_predictions_df > 0].max(axis=1)
    out_of_sample_predictions_df['y_pred'] = out_of_sample_predictions_df[out_of_sample_predictions_df > 0].median(axis=1)
    out_of_sample_predictions_df['y_true'] = dfy_train

    out_of_sample_predictions_df['lower'] = np.exp(out_of_sample_predictions_df['lower'])
    out_of_sample_predictions_df['upper'] = np.exp(out_of_sample_predictions_df['upper'])
    out_of_sample_predictions_df['y_pred'] = np.exp(out_of_sample_predictions_df['y_pred'])
    out_of_sample_predictions_df['y_true'] = np.exp(out_of_sample_predictions_df['y_true'])    

    fig, ax = plt.subplots(figsize=(10, 10))
    
    plt.errorbar(out_of_sample_predictions_df["y_true"], out_of_sample_predictions_df["y_pred"], 
    yerr=(out_of_sample_predictions_df["y_pred"] - out_of_sample_predictions_df["lower"], out_of_sample_predictions_df["upper"] - out_of_sample_predictions_df["y_pred"]),
    ecolor='grey', linestyle='', marker = "o", capsize=5)
    
    plt.plot(plt.xlim(), plt.xlim(), color="lightgray", scalex=False, scaley=False)
    plt.xlabel('Actual Price ($)')
    plt.ylabel('Predicted Price ($)')
    plt.show()

In [ ]:
rep_cross_val_tune_regressor(dfX_train, dfy_train, num_ls, bool_ls, Regressor, param_grid)

In [ ]:
def rep_cross_val_tune_regressor(dfX_train, dfy_train, dfX_test, dfy_test, num_ls, bool_ls, Regressor, param_grid):
    """
    Perform repeated cross-validation for hyperparameter tuning and evaluation of a regression model to come up with prediction intervals,
    and predict on a holdout sample.
    
    Parameters:
    - dfX_train (pd.DataFrame): Training features data.
    - dfy_train (pd.Series or pd.DataFrame): Training target data.
    - dfX_test (pd.DataFrame): Test features data (holdout sample).
    - dfy_test (pd.Series or pd.DataFrame): Test target data (holdout sample).
    - num_ls (list): List of indices or column names for numerical features.
    - bool_ls (list): List of indices or column names for boolean features.
    - Regressor (sklearn estimator): Regressor class to be used.
    - param_grid (dict): Dictionary with parameters names and lists of parameter settings to try.
    
    Returns:
    - out_of_sample_predictions_df (pd.DataFrame): DataFrame containing out-of-sample predictions for each fold.
    - in_sample_metrics (list): List of tuples containing in-sample RMSE and MAE for each fold.
    - out_of_sample_metrics (list): List of tuples containing out-of-sample RMSE and MAE for each fold.
    """
    
    # Define numerical and boolean pipelines
    num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), 
    ('robust', RobustScaler()),
    ('yeojohnson', FunctionTransformer())  # Placeholder for ConditionalTransformer
    ])

    bool_pipeline = Pipeline([
        ('identity', FunctionTransformer())
    ])

    # Combine pipelines with ColumnTransformer
    preprocessor = ColumnTransformer([
        ('num', num_pipeline, num_ls),
        ('bool', bool_pipeline, bool_ls)
    ])

    # Combine preprocessor with model in a pipeline
    model_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', Regressor())
    ])

    # Initialize KFold
    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=0)

    num_folds = 10 * 10  # 10 splits * 10 repeats
    out_of_sample_predictions_df = pd.DataFrame(np.zeros((dfX_train.shape[0], num_folds)))
    holdout_predictions = []

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=model_pipeline, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1, error_score='raise')

    # Perform KFold cross-validation -- score
    for fold_idx, (train_index, test_index) in enumerate(rkf.split(dfX_train)):
        X_train, X_test = dfX_train.iloc[train_index], dfX_train.iloc[test_index]
        y_train, y_test = dfy_train.iloc[train_index], dfy_train.iloc[test_index]

        # Fit GridSearchCV
        grid_search.fit(X_train, y_train)

        # Best parameters
        # print("Best parameters found: ", grid_search.best_params_)

        # Best estimator
        best_model = grid_search.best_estimator_

        # Predict
        y_train_pred = best_model.predict(X_train)
        y_test_pred = best_model.predict(X_test)
        holdout_pred = best_model.predict(dfX_test)

        out_of_sample_predictions_df.iloc[test_index, fold_idx] = y_test_pred
        holdout_predictions.append(holdout_pred[0])

    out_of_sample_predictions_df.columns = [f'y_pred{i+1}' for i in range(num_folds)]

    # Get upper, lower and median from the above table for each record to create prediction intervals
    out_of_sample_predictions_df['lower'] = out_of_sample_predictions_df[out_of_sample_predictions_df > 0].min(axis=1)
    out_of_sample_predictions_df['upper'] = out_of_sample_predictions_df[out_of_sample_predictions_df > 0].max(axis=1)
    out_of_sample_predictions_df['y_pred'] = out_of_sample_predictions_df[out_of_sample_predictions_df > 0].median(axis=1)
    out_of_sample_predictions_df['y_true'] = dfy_train

    out_of_sample_predictions_df['lower'] = np.exp(out_of_sample_predictions_df['lower'])
    out_of_sample_predictions_df['upper'] = np.exp(out_of_sample_predictions_df['upper'])
    out_of_sample_predictions_df['y_pred'] = np.exp(out_of_sample_predictions_df['y_pred'])
    out_of_sample_predictions_df['y_true'] = np.exp(out_of_sample_predictions_df['y_true'])    

    # Process holdout predictions
    holdout_predictions_df = pd.DataFrame([holdout_predictions])
    holdout_predictions_df['lower'] = holdout_predictions_df.min(axis=1)
    holdout_predictions_df['upper'] = holdout_predictions_df.max(axis=1)
    holdout_predictions_df['y_pred'] = holdout_predictions_df.median(axis=1)
    holdout_predictions_df['y_true'] = dfy_test.values[0]

    holdout_predictions_df['lower'] = np.exp(holdout_predictions_df['lower'])
    holdout_predictions_df['upper'] = np.exp(holdout_predictions_df['upper'])
    holdout_predictions_df['y_pred'] = np.exp(holdout_predictions_df['y_pred'])
    holdout_predictions_df['y_true'] = np.exp(holdout_predictions_df['y_true'])

    out_of_sample_predictions_df = pd.concat([out_of_sample_predictions_df, holdout_predictions_df])

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))
    
    plt.errorbar(out_of_sample_predictions_df["y_true"], out_of_sample_predictions_df["y_pred"], 
    yerr=(out_of_sample_predictions_df["y_pred"] - out_of_sample_predictions_df["lower"], out_of_sample_predictions_df["upper"] - out_of_sample_predictions_df["y_pred"]),
    ecolor='grey', linestyle='', marker="o", capsize=5)
    
    # Highlight the holdout prediction
    plt.scatter(holdout_predictions_df['y_true'], holdout_predictions_df['y_pred'], color='red', label='Holdout Sample', zorder=5)
    
    plt.plot(plt.xlim(), plt.xlim(), color="lightgray", scalex=False, scaley=False)
    plt.xlabel('Actual Price ($)')
    plt.ylabel('Predicted Price ($)')
    plt.legend()
    plt.show()


    return holdout_predictions_df

In [ ]:
holdout_predictions_df = rep_cross_val_tune_regressor(dfX_train, dfy_train, dfX_test, dfy_test, num_ls, bool_ls, Regressor, param_grid)

In [ ]:
# the red dot represents the holdout sample "MIN" and the correspondng prediction intervals for the same. For values you can dive into the dataframe below
holdout_predictions_df